# Preprocessamento S4
___

Este notebook realiza um pré-processamento dos dados de S4 disponibilizados em data_s4. O preprocessamento consiste de uma conversão de indices para tratamento de séries temporais, uma reamostragem para amostras de 1s (nos dados orginais, o número de medidas a cada segundo, dependendo do número de satélites visíveis, porém, em geral é mais que um), interpolaçãos spline, seguido de suavização com um kernel gaussiano com janela de tamanho 25, seguindo como uma reamostragem para dados a cada 10 mim, e uma suavização usando um kernel gausiano de janela 25.

In [1]:
import os
import re
import sys

import pathos.multiprocessing as mulprocessing
import pandas as pd

from functools import partial
from scipy.signal import savgol_filter
from pathos.multiprocessing import ProcessPool

import utils
from utils import local_data, local_s4, local_s4_pre
from utils import window

In [2]:
df_station = pd.read_pickle(os.path.join(local_data, 'df_station_sort.pkl'))

In [3]:
files = os.listdir(local_s4)

In [4]:
print(files)

['df_1.pkl.xz', 'df_11.pkl.xz', 'df_14.pkl.xz', 'df_15.pkl.xz', 'df_16.pkl.xz', 'df_17.pkl.xz', 'df_18.pkl.xz', 'df_19.pkl.xz', 'df_20.pkl.xz', 'df_22.pkl.xz', 'df_24.pkl.xz', 'df_25.pkl.xz', 'df_26.pkl.xz', 'df_28.pkl.xz', 'df_29.pkl.xz', 'df_3.pkl.xz', 'df_30.pkl.xz', 'df_31.pkl.xz', 'df_32.pkl.xz', 'df_33.pkl.xz', 'df_34.pkl.xz', 'df_4.pkl.xz', 'df_5.pkl.xz', 'df_6.pkl.xz', 'df_8.pkl.xz', 'df_afl.pkl.xz', 'df_bhz.pkl.xz', 'df_bov.pkl.xz', 'df_bsa.pkl.xz', 'df_cpa.pkl.xz', 'df_cub.pkl.xz', 'df_dou.pkl.xz', 'df_imp.pkl.xz', 'df_ios.pkl.xz', 'df_nta.pkl.xz', 'df_pln.pkl.xz', 'df_pvh.pkl.xz', 'df_sj2.pkl.xz', 'df_sta.pkl.xz', 'df_tfe.pkl.xz']


Inicialmente, os dados de S4 são tais que existe mais de uma amostra por minuto, decorrente dos múltiplos satélites de GPS que estavam no campo de visada da estão naquele intervalo.

O primeiro passo, assim, é combinar as várias medidas no intervalo de minuto gerando uma única medida por minuto, neste caso, tomou-se o valor m?dio de todos as medidas neste intervalo.

O segundo passo é tratar as entradas sem valores, aqui, decidiu-se por utilizar uma interpolação por spline de ordem 3.

O terceiro passo foi uma reamostragem dos dados para intervalores de 10 minutos, de forma, a compatibilizar com as taxas de amostragem dos demais dados a serem utilizados como o VTEC.

Segue-se então como uma suavização do sinal, por meio de uma aplicação de uma média móvel centrada com pesos gaussianos. Note, uma vez que trata-se de uma m?dia centrada:

\begin{equation}
y_t = \sum_{i=-n}^{+n}w_iy_{t+i}
\end{equation}

onde $n$  a srepresenta o tamanho da janela. A suavização definiada, assim, leva em conta pontos futuros, tornando-a inadequada para aplicações em tempo real, pontos no futuro não são conhecidos, e em previsões de séries temporais, a suavização adiciona contribuição do futuro aos dados.

In [5]:
def preprocessing(df_file, window):
    df_s4 = pd.read_pickle(os.path.join(local_s4, df_file), compression='xz')
    df_s4['eventdate'] = pd.to_datetime(df_s4['eventdate'], utc=True)
    df_s4.index = df_s4['eventdate']
    del df_s4['eventdate']
    
    # change for 1s
    df_s4 = df_s4.resample('1T').mean()
    df_s4 = df_s4.interpolate(method='spline', order=4) #4

    # change for 10min
    df_s4 = df_s4.resample('10T').mean()

    # aplay filter Savitzky-Golay
    # aplay a gaussian move average
    df_s4['s4'] = savgol_filter(df_s4.values.ravel(), window, 3)
    df_s4 = df_s4.rolling(window, win_type='gaussian', center=True).mean(std=1.0)
    
    return df_s4

preprocessing_window = partial(preprocessing, window=window)

In [6]:
%%time
df_s4_list = ProcessPool(nodes=mulprocessing.cpu_count()).map(preprocessing_window, files)

CPU times: user 153 ms, sys: 49.9 ms, total: 203 ms
Wall time: 2min 19s


In [7]:
for name, df_s4 in zip(files, df_s4_list):
    df_s4.to_pickle(os.path.join(local_s4_pre,'pre_' + name.replace('.xz', '')))

Contenar os dados de S4 pré-processados.

In [8]:
stations = [i.identificationstation for i in df_station.itertuples()]

In [9]:
def df_list(list_stations, local): 
    for i in list_stations:
        path = local + '/' + 'pre_df_%s.pkl' %i
        df_s4 = pd.read_pickle(path)
        #pattern = re.compile(local + "/" + "pre_df_(.*)\.pkl")
        #name = pattern.sub(r'\1', i)
        yield df_s4.rename(index=str, columns={"s4": i})
        
df_combine = partial(df_list, list_stations=stations, local=local_s4_pre)

df_s4 = pd.concat(df_combine(), axis=1, sort=True)

In [10]:
print(df_s4.shape)
df_s4.head()

(105132, 28)


,afl,bhz,bov,bsa,cpa,32,cub,dou,24,34,...,pln,30,4,pvh,6,26,sta,sj2,29,tfe
2013-01-01 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-01 00:10:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-01 00:20:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-01 00:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-01 00:40:00+00:00,0.20636,NaN,0.29408,NaN,0.223676,NaN,0.189246,0.207889,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.208724,NaN,NaN,NaN


In [11]:
df_s4.index = pd.to_datetime(df_s4.index)


In [12]:
df_s4.to_pickle(os.path.join(local_data, 'df_series_s4.pkl.xz'), compression='xz')

In [13]:
df_s4

,afl,bhz,bov,bsa,cpa,32,cub,dou,24,34,...,pln,30,4,pvh,6,26,sta,sj2,29,tfe
2013-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-01 00:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-01 00:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-01 00:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-01 00:40:00,0.206360,NaN,0.294080,NaN,0.223676,NaN,0.189246,0.207889,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.208724,NaN,NaN,NaN
2013-01-01 00:50:00,0.204201,NaN,0.291502,NaN,0.247244,NaN,0.203963,0.215338,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.205463,NaN,NaN,NaN
2013-01-01 01:00:00,0.199353,NaN,0.284655,NaN,0.277687,NaN,0.215993,0.224347,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.194973,NaN,NaN,NaN
2013-01-01 01:10:00,0.193080,NaN,0.274296,NaN,0.304045,NaN,0.221741,0.234827,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.182419,NaN,NaN,NaN
2013-01-01 01:20:00,0.187098,NaN,0.264138,NaN,0.318292,NaN,0.219275,0.241238,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.175116,NaN,NaN,NaN
2013-01-01 01:30:00,0.182627,NaN,0.253925,NaN,0.321141,NaN,0.209468,0.239972,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.178336,NaN,NaN,NaN
